# Uso de tipos de datos algebraicos genéricos

En este notebook tendrás la oportunidad de prácticas mucho más con los tipos de datos algebraicos genéricos

## Tipos genéricos

Los tipos de datos genéricos no son nuevos, si eres un programador en C++ o aun en Java, has conocido esta asombrosa característica en dichos lenguajes de programación. La idea es que los tipos de datos también pueden tener variables asociadas y en Scala dichas variables se identificarán con letras mayúsculas: `A`, `B`, $\ldots$, y podría ser cualquier nombre en mayúsculas: `ABC`, `IAZ` y así. 

Esas variables de tipos significan, que en un momento determinado cualquier tipo: definido, por definir, etc. Simplemente significa cualquier tipo y puede ser muy útiles cuando queremos significar que una función o un tipo de trabajo pueden trabajar contener o estar asociados con cualquier tipo de dato.

Miraremos los diferentes ejemplos en funciones y en tipos, y al final introduciremos un tema de soslayo (casi lo evitaremos).

### Tipos genéricos en funciones

Muchas funciones pueden trabajar con cualquier tipo. Por ejemplo la famosa función de `id` que recibe una valor de cualquier tipo y retorna el mismo valor (luce uno poco inútil, pero en realidad cuando entremos en temas más profundos veremos que tiene mucha utilidad.

```scala
def id[A](un:A):A = un
```

La anterior función nos dice que estamos recibiendo un valor de cualquier tipo, y que va a retornar dicho valor nuevamente.

Mira ahora otra implementación de la función de `swap` para tuplas binárias:

```scala
def swap[A,B](tpl:(A,B)):(B,A) = (tpl._2, tpl._1)
```

Recuerda que aquí `A` y `B` son cualquier tipo, pero no son de tipo `Any`. Por ejemplo una vez que se invoca la función `swap`, esta asocia `A` y `B` con los correspondiente tipos: 

```scala
swap((true,10)) == (10,true)
```

En la anterior invocación, `A` es asociado al tipo `Boolean` y `B` es asociado al tipo `Int`.

### Tipo genéricos en tipos de datos

Vamos a definir un tipo de dato algebraico que puede contener cualquier tipo de valor, utilizando para ello la construcción por tipo producto:

```scala
sealed trait Tupla
final case class Tupla2[A,B](p:A,s:B) extends Tupla
```

En este caso, el constructor de valores de `Tuple2` para el tipo `Tupla` permite tener cualquier tipo de valor en cada una de sus dos posiciones. Como en siguiente ejemplo:

```scala
val tpl = Tupla2(true,2.0)
```

Este se encarga de construir una tupla `Tupla2[Boolean,Double]`. Los valores de `A` y `B` pueden ser de cualquier tipo y no necesariamente diferente.

```scala
val tpl2 = Tupla2(1,2)
```

### Variante, Covariante e invariantes

Cuando se trabaja con variables de tipos y en particular con la jerarquía de clases que un sistema orientado a objetos como Scala, la pregunta que debes estar haciéndote es que pasa con dicha jerarquía. Para hacerlo muy simple (*puesto que más adelante hablaremos con más detalle de estos temas*), si una Clase `X` tiene subclases a `Y` y `Z`, si tenemos algo genérico `G[A]` que pasa con la relación anteriormente descrita, pues bueno, Scala tiene los término *variante* para indica que si creo una clase de `G[X]`, la relación de subclases se mantedrá con `G[Y]` y `G[Z]` definiendo a `G[+A]`, donde el `+` indica que la relación se mantendrá (*variante*). Si la relación se mantiene como `G[A]` esta es llamada *invariante*, no importa relación alguna. Y, si el valor es `G[-A]` este  indica que el orden de la relación se cambia, si `X` es una superclase de `Y` y `Z`, con `G[-X]` la relación se invierte y esta se convierte en una subclase de `G[Y]` y como también de `G[Z]`, esta se llama *covariante*. 

Te sonará confuso por el momento, pero nosotros te indicaremos cuando debes hacer de una o de la otra.

Otro detalle a tener en cuenta, es que los tipos de datos definidos como `case object` generalmente cuando los tipos son definidos como *variante* se debe tener en encuenta que son los valores bases por lo tanto deben ser definidos como un tipo específico llamado `Nothing`. `Nothing` es un subtipo que pertenece a todos los tipos, es decir que todos los tipos tiene este tipo incluido que es subtipo de todos.

## Ejercicios 

### Ejercicio 1. Tipo identidad.

En este ejercicio, implementarás un tipo algebraico genérico que permita guarda cualquier valor su tipo será `ID` y un constructor de valor llamado `IdE` que te permitirá recibir valores de cualquier tipo. 

>> Implemente el tipo `ID` y su constructor de valor llamado `IdE` e implemente la función `getValue`. 

**Nota** Este tipo es variante.

In [ ]:
// definir tipo ID
// definir constructor de valores de tipo 

def getValue[A](id:ID[A]):A = ???

val un1:ID[Int] = IdE(10)
val un2:ID[Double] = IdE(20.23)
val un3:ID[Boolean] = IdE(true)

getValue(un1) == 10
getValue(un2) == 20.23
getValue(un3) == true

In [ ]:
sealed trait ID[+A]
final case class IdE[+A](value:A) extends ID[A]

def getValue[A](id:ID[A]):A = id match {
    case IdE(a) => a
}

val un1:ID[Int]     = IdE(10)
val un2:ID[Double]  = IdE(20.23)
val un3:ID[Boolean] = IdE(true)

getValue(un1) == 10
getValue(un2) == 20.23
getValue(un3) == true

### Ejercicio 2. Tipo resultado general

Un tipo de dato algebraico muy útil es el expresa una computación que puede fallar, esto se hace interesante dentro de la programación funcional puesto que te permite obviar (*evitar*) el manejo de excepciones (**que son una forma de efecto colateral que debemos evitar**). En la mayoría de los lenguajes funcionales existen dicho tipo, y vamos a implementarlo. De hecho, ya hicimos una versión que permitía tener un valor entero y no tener ninguno, en este caso vamos a generalizar dicho versión.

>> Implementa el tipo `Resultado` y sus dos contructores de valores: `Algun` que recibe un valor de cualquier tipo y `Ningun` que no almacena ningún valor.

**Nota:** Este tipo es variante y ningún extiene `Resultado[Nothing]`.

In [ ]:
// definir el tipo Resultado
// definir el constructor de valor Algun que recibe un valor
// definir el constructor de valor Ningu que no recibe valores

def dividir(a:Int,b:Int):Resultado[Int] = ???

dividir(10,0) == Ningun
dividir(12,2) == Algun(6)
dividir(0,0)  == Ningun

In [ ]:
sealed trait Resultado[+A]
final case class Algun[+A](valor:A) extends Resultado[A]
final case object Ningun  extends Resultado[Nothing]

def dividir(a:Int,b:Int):Resultado[Int] = b match {
    case 0 => Ningun
    case n => Algun(a / n)
}

dividir(10,0) == Ningun
dividir(12,2) == Algun(6)
dividir(0,0)  == Ningun

### Ejercicio 3.  Un par de valores

En este ejercicio te encargarás de crear un versión de tupla, pero la diferencia es que esta versión los valores son del mismo tipo.

>> Crear un tipo llamado `Par` que recibe valores de cualquier tipo y su correspondiente constructor de valor llamado `UnPar` que se encarga de guardar dos valores del mismo tipo. Implemente tambien la función `swap` que recibe un valor de tipo `Par` y crea un nuevo valor con los valores intercambiados.

**Nota:** Este tipo es variante.

In [ ]:
// definir el Tipo Par
// final case class UnPar[+A](p:A, s:A) extends Par[A]

def swap[A](par:Par[A]):Par[A] = ???

val unPar1:Par[Int] = UnPar(1,2)
val unPar2:Par[Int] = UnPar(2,1)
val unPar3:Par[Double] = UnPar(2.3,3.4)
val unPar4:Par[Double] = UnPar(3.4,2.3)

swap(unPar1) == unPar2
swap(unPar3) == unPar4

In [ ]:
sealed trait Par[+A]
final case class UnPar[+A](p:A, s:A) extends Par[A]

def swap[A](par:Par[A]):Par[A] = par match {
    case UnPar(p,s) => UnPar(s,p)
}

val unPar1:Par[Int] = UnPar(1,2)
val unPar2:Par[Int] = UnPar(2,1)
val unPar3:Par[Double] = UnPar(2.3,3.4)
val unPar4:Par[Double] = UnPar(3.4,2.3)

swap(unPar1) == unPar2
swap(unPar3) == unPar4